In [1]:
from dotenv import load_dotenv
from rich import print

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)


True

In [2]:
# cSpell:disable

import re

from langchain.output_parsers import PydanticOutputParser
from langchain_community.document_loaders.web_base import WebBaseLoader
from pydantic import BaseModel, Field

URL = "https://www.agroparistech.fr/formations-ingenieur/mention-biodiversite-ecologie-evolution-montpellier-parcours-m1-biodiversite-vegetale-gestion-ecosystemes-tropicaux-montpellier"

loader = WebBaseLoader(URL)
doc = loader.load()
page = doc[0].page_content.strip()
page_cleaned = re.sub(r"[\t ]+", " ", page)
page_cleaned = re.sub(r"\n{3,}", "\n\n", page_cleaned)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from src.ai_core.llm import get_llm
from src.ai_core.prompts import def_prompt


class Parcours(BaseModel):
    metiers: list[str] = Field(description="liste de metiers auquels peut conduire la formation")
    domaines: list[str] = Field(description="domaines auquels s'applique cette formation")


system = """
    Extraire les metiers et les domaines d'une fiche de description de formation récupérée sur internet.
    Repondre seulement avec les informations disponibles dans la description.
    \n
    """
user = """
    fiche de desctiption de formation:
    ---
    {fiche}
    ---
    {format_instructions}"""

parser = PydanticOutputParser(pydantic_object=Parcours)

prompt = def_prompt(system=system, user=user).partial(format_instructions=parser.get_format_instructions())

llm = get_llm(temperature=0.0)
chain = prompt | llm | parser
summaries = chain.batch([{"fiche": page_cleaned}], {"max_concurrency": 5})

print(summaries)

2025-04-02 15:15:59.690 | INFO     | src.utils.config_mngr:singleton:99 - selected config=training_edenai
2025-04-02 15:15:59.699 | INFO     | src.ai_core.cache:set_method:89 - LLM cache : InMemoryCache
2025-04-02 15:15:59.776 | INFO     | src.ai_core.llm:get_llm:497 - get LLM:'gpt_4omini_edenai' -extra: {'temperature': 0.0}


[
    Parcours(
        metiers=[
            'Chercheur en biodiversité',
            "Gestionnaire d'écosystèmes",
            'Consultant en environnement',
            'Chargé de mission en conservation',
            'Enseignant-chercheur'
        ],
        domaines=[
            'Biodiversité végétale',
            'Écologie',
            'Gestion environnementale',
            'Sciences biologiques',
            'Sciences sociales'
        ]
    )
]